<a href="https://colab.research.google.com/github/iskra3138/colab_repo/blob/master/gradient_in_Eager%26Graph_mode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import numpy as np

## Tensorflow 2.x Style - Eager

In [0]:
model = tf.keras.applications.ResNet101(weights='imagenet')

In [0]:
input = np.ones(shape=(1,224,224,3), dtype=np.float32)

In [0]:
class_idx = np.argmax(model.predict(input))

In [0]:
activation_laver = 'conv5_block3_out'

In [0]:
## get the derivative of y^c w.r.t A^k
y_c = model.layers[-1].output.op.inputs[0][0, class_idx]
A_k = model.get_layer(activation_laver).output

function = tf.keras.models.Model([model.inputs], [A_k, y_c, model.output])
with tf.GradientTape() as tape:
  A_k, y_c, predictions = function(input)
  grad_val_Eager = tape.gradient(y_c, A_k)[0]

conv_output_Eager = A_k[0].numpy()
predictions_Eager = predictions[0].numpy()

weights_Eager = np.mean(grad_val_Eager, axis=(0, 1))  
gradcam_Eager = np.dot(conv_output_Eager, weights_Eager) 

## Relu
gradcam_Eager = np.maximum(gradcam_Eager, 0)

In [8]:
gradcam_Eager

array([[0.00028162, 0.0007245 , 0.00046242, 0.00021268, 0.00021519,
        0.0003005 , 0.00063932],
       [0.00066276, 0.00067729, 0.0007408 , 0.00102169, 0.00093498,
        0.00044946, 0.00041186],
       [0.00056785, 0.00162363, 0.0050569 , 0.00652022, 0.00586844,
        0.0030137 , 0.00104003],
       [0.00069998, 0.00360565, 0.01008291, 0.01137574, 0.00987676,
        0.00523698, 0.00245925],
       [0.00082678, 0.00376313, 0.01053694, 0.01221692, 0.01008545,
        0.00563704, 0.00339998],
       [0.00080959, 0.0023342 , 0.00609587, 0.00825931, 0.00669402,
        0.00456461, 0.00227837],
       [0.00066882, 0.00133016, 0.00171012, 0.00234924, 0.00237031,
        0.00177611, 0.00059871]], dtype=float32)

In [9]:
conv_output_Eager[:,:,0]

array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.9376273, 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ]], dtype=float32)

In [10]:
predictions_Eager[class_idx-2:class_idx+3]

array([1.68166967e-04, 3.86879401e-05, 5.93045726e-02, 1.05256186e-04,
       1.13464345e-03], dtype=float32)

## Tensorflow 1.x Style - Graph

In [11]:
tf.compat.v1.disable_eager_execution()
model = tf.keras.applications.ResNet101(weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
## get the derivative of y^c w.r.t A^k
y_c = model.layers[-1].output.op.inputs[0][0, class_idx]
A_k = model.get_layer(activation_laver).output

grad_val_Graph = tf.gradients(y_c, A_k)[0]
gradient_fn = tf.keras.backend.function([model.input], [A_k, grad_val_Graph, model.output])

conv_output_Graph, grad_val_Graph, predictions_Graph = gradient_fn([input])
conv_output_Graph, grad_val_Graph, predictions_Graph = conv_output_Graph[0], grad_val_Graph[0], predictions_Graph[0]

weights_Graph = np.mean(grad_val_Graph, axis=(0, 1))
gradcam_Graph = np.dot(conv_output_Graph, weights_Graph)

## Relu
gradcam_Graph = np.maximum(gradcam_Graph, 0)

In [13]:
gradcam_Graph

array([[0.0058591 , 0.02567193, 0.01936997, 0.0137712 , 0.0128011 ,
        0.0129117 , 0.02226914],
       [0.02281783, 0.02571142, 0.03634336, 0.0456591 , 0.04175134,
        0.02390287, 0.02082711],
       [0.02060051, 0.06106661, 0.16102049, 0.20707057, 0.18705162,
        0.10284157, 0.03869095],
       [0.03114092, 0.1264236 , 0.294668  , 0.3358269 , 0.2971611 ,
        0.17591155, 0.0738475 ],
       [0.03364016, 0.1308052 , 0.30485022, 0.35737312, 0.30658722,
        0.1922845 , 0.10127223],
       [0.02695532, 0.08306788, 0.19243468, 0.2523846 , 0.21743369,
        0.14924252, 0.07038921],
       [0.02708533, 0.04796556, 0.06668366, 0.0885573 , 0.08739071,
        0.06345784, 0.03115967]], dtype=float32)

In [14]:
conv_output_Graph[:,:,0]

array([[0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.9376209, 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       ]], dtype=float32)

In [15]:
predictions_Graph[class_idx-2:class_idx+3]

array([1.6816732e-04, 3.8687980e-05, 5.9304491e-02, 1.0525640e-04,
       1.1346451e-03], dtype=float32)

# TF2.0 Style(Eager mode) &  TF1.0 Style(Graph mode) 

### GRADCAM

In [16]:
gradcam_Graph - gradcam_Eager

array([[0.00557748, 0.02494742, 0.01890755, 0.01355851, 0.01258591,
        0.0126112 , 0.02162982],
       [0.02215507, 0.02503413, 0.03560256, 0.04463741, 0.04081637,
        0.02345341, 0.02041526],
       [0.02003266, 0.05944298, 0.15596358, 0.20055035, 0.18118319,
        0.09982788, 0.03765092],
       [0.03044094, 0.12281795, 0.2845851 , 0.32445115, 0.28728434,
        0.17067456, 0.07138826],
       [0.03281339, 0.12704206, 0.29431328, 0.3451562 , 0.29650176,
        0.18664746, 0.09787225],
       [0.02614573, 0.08073368, 0.18633881, 0.24412529, 0.21073967,
        0.1446779 , 0.06811085],
       [0.02641651, 0.0466354 , 0.06497354, 0.08620807, 0.08502039,
        0.06168173, 0.03056096]], dtype=float32)

In [17]:
np.max(gradcam_Graph - gradcam_Eager)

0.3451562

### A_k output

In [29]:
(conv_output_Graph - conv_output_Eager)[:,:,450]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -4.05311584e-06,
        -9.53674316e-06, -9.29832458e-06, -8.64267349e-06,
         0.00000000e+00],
       [ 0.00000000e+00, -3.33786011e-06, -7.62939453e-06,
        -1.04904175e-05, -1.90734863e-05, -1.52587891e-05,
         0.00000000e+00],
       [ 0.00000000e+00, -3.33786011e-06, -1.28746033e-05,
        -1.81198120e-05, -2.43186951e-05, -1.09672546e-05,
        -1.60932541e-06],
       [ 0.00000000e+00, -6.85453415e-07, -1.66893005e-06,
        -1.90734863e-05, -1.95503235e-05, -4.41074371e-06,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        -3.09944153e-06, -2.38418579e-06,  0.00000000e+00,
         0.0000000

In [18]:
np.max((conv_output_Graph - conv_output_Eager))

3.0040741e-05

### Predictions

In [19]:
(predictions_Graph - predictions_Eager)[class_idx-2:class_idx+3]

array([ 3.4924597e-10,  4.0017767e-11, -8.1956387e-08,  2.1100277e-10,
        1.6298145e-09], dtype=float32)

In [20]:
np.max(predictions_Graph - predictions_Eager)

2.0489097e-08